# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Time_Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit 20dbcd20516269f15905cee3336a9034b1e6e722
Author: Najwa Ez Zine <nezzine@fnantes-stretch>
Date:   Wed May 30 15:53:55 2018 +0200

    Cleaning


Python 2.7.13


#### Préparation de l'environnement

In [3]:
%%bash -s
pip3 install --user execo
pip3 install --user requests

  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a94236f6d4b8223e21beb4d576dc72e8130bd7880f600839b8/urllib3-1.22-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl


In [4]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output

##### Réservation noeuds

In [5]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 4
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster != 'econome'\"" 

# Nancy
site = "nantes"

In [37]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 91609


On les range

In [38]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[]

##### Déploiement de l'environnement

In [8]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [9]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

nantes: Deployment #D-1cc1afe0-fd8b-48ab-aefd-91d2ad42cc44 started
nantes: Grab the key file /home/nezzine/.ssh/authorized_keys
nantes: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
nantes: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
nantes: Launching a deployment on ecotype-[7,9,44,47].nantes.grid5000.fr
nantes: Performing a Deploy[SetDeploymentEnvUntrusted] step
nantes:   switch_pxe
nantes:   reboot
nantes:    * Performing a soft reboot on ecotype-[7,9,44,47].nantes.grid5000.fr
nantes:   wait_reboot
nantes:   send_key_in_deploy_env
nantes:   create_partition_table
nantes:   format_deploy_part
nantes:   mount_deploy_part
nantes:   format_tmp_part
nantes:   format_swap_part
nantes: End of step Deploy[SetDeploymentEnvUntrusted] after 184s
nantes: Performing a Deploy[BroadcastEnvKascade] step
nantes:   send_environment
nantes:    * Broadcast time: 104s
nantes:   manage_admin_post_install
nantes:   mana

nantes: Warning: Permanently added 'nantes.grid5000.fr,172.16.207.100' (ECDSA) to the list of known hosts.
nantes: Connection to nantes.grid5000.fr closed.


#### Test : commande simple 'ls'

In [ ]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

#### Installation environnement sur noeud

In [10]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [11]:
Remote_install.run().ok

True

#### Version Simgrid

In [12]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [13]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [14]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [15]:
# Converting and organizing data
def tab_conv(l,NPROCS) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        S[int(l[i].split()[1])] = int(float(l[i].split()[5]))
    return S

In [16]:
def delayer(num_node,NPROCS,date):
    D = init_tab(NPROCS,0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS)
    
    for p in range(0, NPROCS):
        D[p] = S[p] - min(S)
    return D

In [ ]:
delayer(1,2,'2018-05-30_12:19')

In [17]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

##### Shortcut

In [23]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
def parl_exec () :
    Result_get.run().ok
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

# Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10',hosts=nodes,connection_params={'user':'root'})



Src_get.run().ok
Remote_smpi_env.run().ok

# Remote_smpi.run().ok

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

2018-05-31_11:01
ecotype-44 => 0 NPROCS = 10 MIN = 0 MAX = 25 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[6, 6, 9, 3, 0, 11, 7, 2, 7, 10]
ecotype-47 => 1 NPROCS = 10 MIN = 25 MAX = 50 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[8, 0, 6, 6, 6, 6, 6, 8, 9, 5]
ecotype-7 => 2 NPROCS = 10 MIN = 50 MAX = 75 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[6, 0, 6, 6, 6, 8, 9, 7, 8, 4]
ecotype-9 => 3 NPROCS = 10 MIN = 75 MAX = 100 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### //

In [36]:
import random
import sys
from threading import Thread
import time


Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))



Src_get.run().ok
Remote_smpi_env.run().ok


machines = init_tab(len(nodes),None)

class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""

    def __init__(self, MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
       
                      
            
# Création des threads
Result_get.run().ok
# Nbr pcs
NPROCS = 10
    
# Nbr itérations
N = 100
    
D = init_tab(NPROCS,0)
delay = ' '.join(str(e) for e in D)
date = datetime.datetime.now().strftime('%Y-%m-%d');
date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')



for i in range (0,nodecount) :
    MIN = (int)((i*N)/len(nodes))
    MAX = (int)(((i+1)*N)/len(nodes)) 
    machines[i] = ParSimer(MIN,MAX,N,delay)

# Lancement des threads
for i in range (0,nodecount) :
    machines[i].start()

for i in range (0,nodecount) :
    machines[i].join()
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok     
     print(delayer(i,NPROCS,date_precise))

2018-05-31 11:57:25,336 WARNING: terminated: <SshProcess('mkdir -p experiments_src', Host('ecotype-47.nantes.grid5000.fr'), connection_params={'user': 'root'}, name=mkdir -p experiments_src, started=True, start_date=2018-05-31 11:57:06+02:00, ended=True, end_date=2018-05-31 11:57:25+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=65280, ok=False, pid=2915, real cmd=('ssh', '-tt', '-o', 'BatchMode=yes', '-o', 'PasswordAuthentication=no', '-o', 'StrictHostKeyChecking=no', '-o', 'UserKnownHostsFile=/dev/null', '-o', 'ConnectTimeout=20', '-o', 'User=root', 'ecotype-47.nantes.grid5000.fr', 'mkdir -p experiments_src'))>
stdout:

stderr:
ssh: connect to host ecotype-47.nantes.grid5000.fr port 22: No route to host

2018-05-31 11:57:25,339 WARNING: terminated: <SshProcess('mkdir -p experiments_src', Host('ecotype-44.nantes.grid5000.fr'), connection_params={'user': 'root'}, name=mkdir -p experiments_src, started=True, start_da

False
2018-05-31 11:57:49,052 WARNING: terminated: <SshProcess("cd ~/experiments_src && make && make run NPROCS=10 MIN=0 MAX=25 N=100 DELAY='0 0 0 0 0 0 0 0 0 0'", Host('ecotype-44.nantes.grid5000.fr'), connection_params={'user': 'root'}, name=cd ~/experiments_src && make && make..., started=True, start_date=2018-05-31 11:57:45+02:00, ended=True, end_date=2018-05-31 11:57:49+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=65280, ok=False, pid=3771, real cmd=('ssh', '-tt', '-o', 'BatchMode=yes', '-o', 'PasswordAuthentication=no', '-o', 'StrictHostKeyChecking=no', '-o', 'UserKnownHostsFile=/dev/null', '-o', 'ConnectTimeout=20', '-o', 'User=root', 'ecotype-44.nantes.grid5000.fr', "cd ~/experiments_src && make && make run NPROCS=10 MIN=0 MAX=25 N=100 DELAY='0 0 0 0 0 0 0 0 0 0'"))>
stdout:

stderr:
ssh: connect to host ecotype-44.nantes.grid5000.fr port 22: No route to host

False
2018-05-31 11:57:49,096 WARNING: termina

Exception in thread Thread-51:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "<ipython-input-36-02753e9833c2>", line 53, in run
    print(delayer(i,NPROCS,date_precise))
  File "<ipython-input-16-2434d7dba9f7>", line 4, in delayer
    raw_data = parser(num_node,date)
  File "<ipython-input-14-99b22b4490ea>", line 7, in parser
    + '.txt') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/2018-05-31/ecotype-44_2018-05-31_11:57.txt'

Exception in thread Thread-48:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "<ipython-input-36-02753e9833c2>", line 53, in run
    print(delayer(i,NPROCS,date_precise))
  File "<ipython-input-16-2434d7dba9f7>", line 4, in delayer
    raw_data = parser(num_node,date)
  File "<ipython-input-14-99b22b4490ea>", line 

2018-05-31 11:57:52,177 WARNING: terminated: <Process('scp -o BatchMode=yes -o PasswordAuthentication=no -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -o ConnectTimeout=20 -rp -o User=root ecotype-44.nantes.grid5000.fr:~/experiments_src/bcast_results.txt ~/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/2018-05-31/ecotype-44_2018-05-31_11:57.txt', shell=True, name=scp -o BatchMode=yes -o PasswordAuth..., started=True, start_date=2018-05-31 11:57:49+02:00, ended=True, end_date=2018-05-31 11:57:52+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=256, ok=False, pid=3829)>
stdout:

stderr:
ssh: connect to host ecotype-44.nantes.grid5000.fr port 22: No route to host

2018-05-31 11:57:55,303 WARNING: terminated: <Process('scp -o BatchMode=yes -o PasswordAuthentication=no -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -o ConnectTimeout=20 -rp -o User=root ecotype-47.nantes.grid5000.fr:~/expe

#### Publication des résultats

In [ ]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2"' Delayer'

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)